# Временные ряды и рекуррентные нейронные сети (LSTM/GRU)

В этом ноутбуке работаем с **реальными временными рядами** и применяем:
- **Классические методы**: ARIMA, SARIMA, Prophet
- **Глубокое обучение**: LSTM, GRU, Bidirectional LSTM

## Датасеты

1. **Airline Passengers** - классический временной ряд с трендом и сезонностью
2. **Stock Prices (Apple)** - финансовые данные с высокой волатильностью
3. **Weather Data (Temperature)** - мультивариатные временные ряды

## Содержание

### Часть 1: Airline Passengers (Univariate)
1. Загрузка и EDA
2. Декомпозиция временного ряда
3. ARIMA/SARIMA
4. Prophet
5. LSTM
6. GRU
7. Сравнение методов

### Часть 2: Stock Prices (Multivariate)
8. Загрузка финансовых данных
9. Технические индикаторы
10. LSTM для предсказания цен
11. Attention механизм

### Часть 3: Weather Data (Multivariate)
12. Мультивариатные временные ряды
13. Bidirectional LSTM
14. Seq2Seq архитектура

In [ ]:
# Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Временные ряды - классические методы
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Bidirectional

# Sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Утилиты
from datetime import datetime, timedelta
import joblib

# Настройка отображения
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Seed для воспроизводимости
np.random.seed(42)
tf.random.set_seed(42)

print('✅ Все библиотеки загружены!')
print(f'TensorFlow: {tf.__version__}')
print(f'Prophet доступен: True')

---
# ЧАСТЬ 1: Airline Passengers (Univariate Time Series)

Классический датасет с количеством авиапассажиров с 1949 по 1960 год.

## 1.1 Загрузка и исследовательский анализ

In [ ]:
# Загрузка данных Airline Passengers
# Этот датасет встроен в seaborn
df_flights = sns.load_dataset('flights')

# Преобразуем в временной ряд
df_flights['date'] = pd.to_datetime(df_flights['year'].astype(str) + '-' + df_flights['month'], format='%Y-%B')
df_flights = df_flights.sort_values('date').reset_index(drop=True)
df_flights = df_flights.set_index('date')

print(f'Датасет: {df_flights.shape}')
print(f'Период: {df_flights.index.min()} - {df_flights.index.max()}')
print(f'Частота: Monthly')
print(f'\nПервые строки:')
df_flights.head(10)

In [ ]:
# Визуализация временного ряда
fig, axes = plt.subplots(2, 1, figsize=(15, 8))

# Полный ряд
axes[0].plot(df_flights.index, df_flights['passengers'], linewidth=2)
axes[0].set_title('Airline Passengers (1949-1960)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Дата')
axes[0].set_ylabel('Количество пассажиров')
axes[0].grid(True, alpha=0.3)

# Распределение
axes[1].hist(df_flights['passengers'], bins=30, edgecolor='black', alpha=0.7)
axes[1].set_title('Распределение значений', fontsize=12)
axes[1].set_xlabel('Количество пассажиров')
axes[1].set_ylabel('Частота')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print('\n📊 Наблюдения:')
print('1. Явный восходящий тренд (рост количества пассажиров)')
print('2. Сезонность (повторяющиеся паттерны каждый год)')
print('3. Увеличение амплитуды колебаний со временем (гетероскедастичность)')

## 1.2 Декомпозиция временного ряда

Разложим ряд на компоненты: **тренд, сезонность, остатки**

In [ ]:
# Декомпозиция (additive и multiplicative)
decomposition_add = seasonal_decompose(df_flights['passengers'], model='additive', period=12)
decomposition_mult = seasonal_decompose(df_flights['passengers'], model='multiplicative', period=12)

# Визуализация
fig, axes = plt.subplots(4, 2, figsize=(16, 12))
fig.suptitle('Декомпозиция временного ряда', fontsize=16, fontweight='bold')

# Additive
axes[0, 0].plot(df_flights['passengers'])
axes[0, 0].set_title('Оригинальный ряд')
axes[0, 0].set_ylabel('Пассажиры')

axes[1, 0].plot(decomposition_add.trend)
axes[1, 0].set_title('Тренд (Additive)')
axes[1, 0].set_ylabel('Тренд')

axes[2, 0].plot(decomposition_add.seasonal)
axes[2, 0].set_title('Сезонность (Additive)')
axes[2, 0].set_ylabel('Сезонность')

axes[3, 0].plot(decomposition_add.resid)
axes[3, 0].set_title('Остатки (Additive)')
axes[3, 0].set_ylabel('Остатки')
axes[3, 0].set_xlabel('Дата')

# Multiplicative
axes[0, 1].plot(df_flights['passengers'])
axes[0, 1].set_title('Оригинальный ряд')

axes[1, 1].plot(decomposition_mult.trend)
axes[1, 1].set_title('Тренд (Multiplicative)')

axes[2, 1].plot(decomposition_mult.seasonal)
axes[2, 1].set_title('Сезонность (Multiplicative)')

axes[3, 1].plot(decomposition_mult.resid)
axes[3, 1].set_title('Остатки (Multiplicative)')
axes[3, 1].set_xlabel('Дата')

plt.tight_layout()
plt.show()

print('\n📈 Интерпретация:')
print('- Additive: Y(t) = Trend(t) + Seasonal(t) + Residual(t)')
print('- Multiplicative: Y(t) = Trend(t) × Seasonal(t) × Residual(t)')
print('\nДля данного ряда лучше подходит multiplicative, т.к. амплитуда сезонности растет со временем')

## 1.3 Тест на стационарность (ADF Test)

In [ ]:
# Augmented Dickey-Fuller Test
def adf_test(series, name=''):
    result = adfuller(series.dropna())
    print(f'\n=== ADF Test: {name} ===')
    print(f'ADF Statistic: {result[0]:.6f}')
    print(f'p-value: {result[1]:.6f}')
    print(f'Critical Values:')
    for key, value in result[4].items():
        print(f'  {key}: {value:.3f}')
    
    if result[1] <= 0.05:
        print('✅ Ряд стационарен (отвергаем H0)')
    else:
        print('❌ Ряд НЕ стационарен (не можем отвергнуть H0)')
    
    return result[1] <= 0.05

# Тест оригинального ряда
is_stationary = adf_test(df_flights['passengers'], 'Оригинальный ряд')

# Тест после дифференцирования
df_flights['passengers_diff'] = df_flights['passengers'].diff()
is_stationary_diff = adf_test(df_flights['passengers_diff'], 'После 1-го дифференцирования')

# Тест после сезонного дифференцирования
df_flights['passengers_seasonal_diff'] = df_flights['passengers'].diff(12)
is_stationary_seasonal = adf_test(df_flights['passengers_seasonal_diff'], 'После сезонного дифференцирования')

## 1.4 Разделение на train/test

In [ ]:
# Разделяем: 80% train, 20% test
train_size = int(len(df_flights) * 0.8)
train_flights = df_flights['passengers'][:train_size]
test_flights = df_flights['passengers'][train_size:]

print(f'Train: {len(train_flights)} наблюдений ({train_flights.index.min()} - {train_flights.index.max()})')
print(f'Test: {len(test_flights)} наблюдений ({test_flights.index.min()} - {test_flights.index.max()})')

# Визуализация разделения
plt.figure(figsize=(15, 5))
plt.plot(train_flights.index, train_flights, label='Train', linewidth=2)
plt.plot(test_flights.index, test_flights, label='Test', linewidth=2, color='orange')
plt.axvline(x=train_flights.index[-1], color='red', linestyle='--', label='Train/Test Split')
plt.title('Train/Test Split', fontsize=14)
plt.xlabel('Дата')
plt.ylabel('Пассажиры')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 1.5 SARIMA - Сезонная ARIMA

SARIMA(p, d, q)(P, D, Q, s) где:
- p, d, q - порядки AR, I, MA
- P, D, Q - сезонные порядки
- s - период сезонности (12 для месячных данных)

In [ ]:
# Обучение SARIMA модели
# Параметры подобраны вручную (можно использовать auto_arima для автоподбора)
print('Обучаем SARIMA(1,1,1)(1,1,1,12)...')

sarima_model = SARIMAX(
    train_flights,
    order=(1, 1, 1),  # (p, d, q)
    seasonal_order=(1, 1, 1, 12),  # (P, D, Q, s)
    enforce_stationarity=False,
    enforce_invertibility=False
)

sarima_fit = sarima_model.fit(disp=False)
print('✅ SARIMA модель обучена!')
print(sarima_fit.summary())

In [ ]:
# Предсказания SARIMA
sarima_forecast = sarima_fit.forecast(steps=len(test_flights))
sarima_forecast_series = pd.Series(sarima_forecast.values, index=test_flights.index)

# Метрики
sarima_mae = mean_absolute_error(test_flights, sarima_forecast)
sarima_rmse = np.sqrt(mean_squared_error(test_flights, sarima_forecast))
sarima_mape = np.mean(np.abs((test_flights - sarima_forecast) / test_flights)) * 100

print('=== SARIMA Results ===')
print(f'MAE: {sarima_mae:.2f}')
print(f'RMSE: {sarima_rmse:.2f}')
print(f'MAPE: {sarima_mape:.2f}%')

# Визуализация
plt.figure(figsize=(15, 6))
plt.plot(train_flights.index, train_flights, label='Train', linewidth=2)
plt.plot(test_flights.index, test_flights, label='Test (Actual)', linewidth=2, color='orange')
plt.plot(sarima_forecast_series.index, sarima_forecast_series, label='SARIMA Forecast', linewidth=2, color='green', linestyle='--')
plt.title('SARIMA Forecast', fontsize=14)
plt.xlabel('Дата')
plt.ylabel('Пассажиры')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 1.6 Prophet - Автоматическое прогнозирование от Facebook

In [ ]:
# Подготовка данных для Prophet (требует колонки 'ds' и 'y')
train_prophet = train_flights.reset_index()
train_prophet.columns = ['ds', 'y']

# Создание и обучение модели Prophet
print('Обучаем Prophet модель...')
prophet_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False,
    seasonality_mode='multiplicative'
)

prophet_model.fit(train_prophet)
print('✅ Prophet модель обучена!')

In [ ]:
# Прогноз Prophet
future = prophet_model.make_future_dataframe(periods=len(test_flights), freq='MS')
prophet_forecast = prophet_model.predict(future)

# Извлекаем прогноз для test периода
prophet_test_forecast = prophet_forecast.iloc[-len(test_flights):]['yhat'].values

# Метрики
prophet_mae = mean_absolute_error(test_flights, prophet_test_forecast)
prophet_rmse = np.sqrt(mean_squared_error(test_flights, prophet_test_forecast))
prophet_mape = np.mean(np.abs((test_flights - prophet_test_forecast) / test_flights)) * 100

print('=== Prophet Results ===')
print(f'MAE: {prophet_mae:.2f}')
print(f'RMSE: {prophet_rmse:.2f}')
print(f'MAPE: {prophet_mape:.2f}%')

# Визуализация Prophet
fig = prophet_model.plot(prophet_forecast)
plt.title('Prophet Forecast', fontsize=14)
plt.tight_layout()
plt.show()

# Компоненты Prophet
fig2 = prophet_model.plot_components(prophet_forecast)
plt.tight_layout()
plt.show()

## 1.7 LSTM - Long Short-Term Memory

In [ ]:
# Подготовка данных для LSTM
def create_sequences(data, seq_length):
    """Создает последовательности для обучения LSTM"""
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Нормализация данных
scaler = MinMaxScaler()
passengers_scaled = scaler.fit_transform(df_flights[['passengers']])

# Разделение
train_scaled = passengers_scaled[:train_size]
test_scaled = passengers_scaled[train_size:]

# Создание последовательностей (используем 12 месяцев для предсказания следующего)
seq_length = 12
X_train, y_train = create_sequences(train_scaled, seq_length)
X_test, y_test = create_sequences(test_scaled, seq_length)

# Reshape для LSTM [samples, time steps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

print(f'✅ Данные подготовлены для LSTM')
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')

In [ ]:
# Создание LSTM модели
def create_lstm_model(input_shape):
    model = models.Sequential([
        LSTM(50, activation='relu', return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50, activation='relu'),
        Dropout(0.2),
        Dense(1)
    ])
    
    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae']
    )
    
    return model

lstm_model = create_lstm_model((seq_length, 1))
print('=== LSTM Architecture ===')
lstm_model.summary()

In [ ]:
# Обучение LSTM
print('Обучаем LSTM...')

early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-7)

history_lstm = lstm_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=16,
    callbacks=[early_stop, reduce_lr],
    verbose=0
)

print('✅ LSTM обучена!')

# Визуализация обучения
plt.figure(figsize=(12, 4))
plt.plot(history_lstm.history['loss'], label='Train Loss')
plt.plot(history_lstm.history['val_loss'], label='Val Loss')
plt.title('LSTM Training History')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Предсказания LSTM
lstm_predictions_scaled = lstm_model.predict(X_test)
lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

# Метрики
lstm_mae = mean_absolute_error(y_test_actual, lstm_predictions)
lstm_rmse = np.sqrt(mean_squared_error(y_test_actual, lstm_predictions))
lstm_mape = np.mean(np.abs((y_test_actual - lstm_predictions) / y_test_actual)) * 100

print('=== LSTM Results ===')
print(f'MAE: {lstm_mae:.2f}')
print(f'RMSE: {lstm_rmse:.2f}')
print(f'MAPE: {lstm_mape:.2f}%')

# Визуализация
plt.figure(figsize=(15, 6))
plt.plot(range(len(y_test_actual)), y_test_actual, label='Actual', linewidth=2)
plt.plot(range(len(lstm_predictions)), lstm_predictions, label='LSTM Predictions', linewidth=2, linestyle='--')
plt.title('LSTM Predictions vs Actual', fontsize=14)
plt.xlabel('Time Step')
plt.ylabel('Passengers')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 1.8 GRU - Gated Recurrent Unit

In [ ]:
# Создание GRU модели
def create_gru_model(input_shape):
    model = models.Sequential([
        GRU(50, activation='relu', return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        GRU(50, activation='relu'),
        Dropout(0.2),
        Dense(1)
    ])
    
    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae']
    )
    
    return model

gru_model = create_gru_model((seq_length, 1))
print('=== GRU Architecture ===')
gru_model.summary()

In [ ]:
# Обучение GRU
print('Обучаем GRU...')

history_gru = gru_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=16,
    callbacks=[early_stop, reduce_lr],
    verbose=0
)

print('✅ GRU обучена!')

# Предсказания GRU
gru_predictions_scaled = gru_model.predict(X_test)
gru_predictions = scaler.inverse_transform(gru_predictions_scaled)

# Метрики
gru_mae = mean_absolute_error(y_test_actual, gru_predictions)
gru_rmse = np.sqrt(mean_squared_error(y_test_actual, gru_predictions))
gru_mape = np.mean(np.abs((y_test_actual - gru_predictions) / y_test_actual)) * 100

print('=== GRU Results ===')
print(f'MAE: {gru_mae:.2f}')
print(f'RMSE: {gru_rmse:.2f}')
print(f'MAPE: {gru_mape:.2f}%')

## 1.9 Сравнение всех методов для Airline Passengers

In [ ]:
# Сводная таблица результатов
results_airline = pd.DataFrame({
    'Method': ['SARIMA', 'Prophet', 'LSTM', 'GRU'],
    'MAE': [sarima_mae, prophet_mae, lstm_mae, gru_mae],
    'RMSE': [sarima_rmse, prophet_rmse, lstm_rmse, gru_rmse],
    'MAPE (%)': [sarima_mape, prophet_mape, lstm_mape, gru_mape]
})

results_airline = results_airline.sort_values('RMSE')
print('\n=== Сравнение методов: Airline Passengers ===')
print(results_airline.to_string(index=False))

# Визуализация сравнения
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(results_airline))
width = 0.25

ax.bar(x - width, results_airline['MAE'], width, label='MAE')
ax.bar(x, results_airline['RMSE'], width, label='RMSE')
ax.bar(x + width, results_airline['MAPE (%)'], width, label='MAPE (%)')

ax.set_xlabel('Метод', fontsize=12)
ax.set_ylabel('Значение метрики', fontsize=12)
ax.set_title('Сравнение методов прогнозирования', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(results_airline['Method'])
ax.legend()
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print('\n🏆 Лучший метод:', results_airline.iloc[0]['Method'])

---
# ЧАСТЬ 2: Stock Prices (Multivariate Time Series)

Предсказание цен акций с использованием множества признаков.

In [ ]:
# Загрузка данных акций (используем yfinance или создадим синтетические данные)
print('\n📈 Часть 2: Stock Prices')
print('Для демонстрации создаем синтетические данные, похожие на реальные акции')
print('В реальности используйте yfinance для загрузки: yf.download("AAPL", start="2020-01-01")')

# Создаем синтетические данные акций
np.random.seed(42)
dates = pd.date_range(start='2020-01-01', end='2023-12-31', freq='D')

# Генерируем цены с трендом и волатильностью
trend = np.linspace(100, 200, len(dates))
noise = np.random.normal(0, 5, len(dates))
seasonality = 10 * np.sin(np.arange(len(dates)) / 365 * 2 * np.pi)
close_price = trend + noise + seasonality

# Создаем OHLC данные
df_stock = pd.DataFrame({
    'Date': dates,
    'Open': close_price + np.random.uniform(-2, 2, len(dates)),
    'High': close_price + np.random.uniform(0, 5, len(dates)),
    'Low': close_price - np.random.uniform(0, 5, len(dates)),
    'Close': close_price,
    'Volume': np.random.randint(1000000, 10000000, len(dates))
})

df_stock = df_stock.set_index('Date')

print(f'\nДатасет: {df_stock.shape}')
print(f'Период: {df_stock.index.min()} - {df_stock.index.max()}')
df_stock.head(10)

In [ ]:
# Визуализация акций
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Цена закрытия
axes[0].plot(df_stock.index, df_stock['Close'], linewidth=1, alpha=0.8)
axes[0].set_title('Stock Price (Close)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Price ($)')
axes[0].grid(True, alpha=0.3)

# Объем торгов
axes[1].bar(df_stock.index, df_stock['Volume'], width=1, alpha=0.7)
axes[1].set_title('Trading Volume', fontsize=12)
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Volume')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Технические индикаторы
def add_technical_indicators(df):
    """Добавляет технические индикаторы для анализа акций"""
    # Moving Averages
    df['MA_7'] = df['Close'].rolling(window=7).mean()
    df['MA_21'] = df['Close'].rolling(window=21).mean()
    df['MA_50'] = df['Close'].rolling(window=50).mean()
    
    # Относительная доходность
    df['Returns'] = df['Close'].pct_change()
    
    # Volatility (rolling std)
    df['Volatility'] = df['Returns'].rolling(window=21).std()
    
    # Momentum
    df['Momentum'] = df['Close'] - df['Close'].shift(10)
    
    # RSI (Relative Strength Index)
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    return df

df_stock = add_technical_indicators(df_stock)
df_stock = df_stock.dropna()  # Удаляем NaN от rolling операций

print('✅ Технические индикаторы добавлены!')
print(f'Признаки: {list(df_stock.columns)}')
df_stock.tail()

In [ ]:
# Подготовка данных для LSTM (multivariate)
feature_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'MA_7', 'MA_21', 'Returns', 'RSI']
target_column = 'Close'

# Нормализация
scaler_stock = MinMaxScaler()
stock_scaled = scaler_stock.fit_transform(df_stock[feature_columns])

# Разделение на train/test (80/20)
train_size_stock = int(len(stock_scaled) * 0.8)
train_stock = stock_scaled[:train_size_stock]
test_stock = stock_scaled[train_size_stock:]

# Создание последовательностей (60 дней для предсказания следующего)
seq_length_stock = 60
X_train_stock, y_train_stock = create_sequences(train_stock[:, 3], seq_length_stock)  # Используем Close
X_test_stock, y_test_stock = create_sequences(test_stock[:, 3], seq_length_stock)

X_train_stock = X_train_stock.reshape((X_train_stock.shape[0], X_train_stock.shape[1], 1))
X_test_stock = X_test_stock.reshape((X_test_stock.shape[0], X_test_stock.shape[1], 1))

print(f'✅ Данные акций подготовлены')
print(f'X_train shape: {X_train_stock.shape}')
print(f'X_test shape: {X_test_stock.shape}')

In [ ]:
# LSTM для предсказания цен акций
stock_lstm_model = create_lstm_model((seq_length_stock, 1))

print('Обучаем LSTM для акций...')
history_stock = stock_lstm_model.fit(
    X_train_stock, y_train_stock,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, reduce_lr],
    verbose=0
)

print('✅ LSTM для акций обучена!')

# Предсказания
stock_predictions = stock_lstm_model.predict(X_test_stock)

# Денормализация (только для Close price)
close_scaler = MinMaxScaler()
close_scaler.fit(df_stock[['Close']][:train_size_stock])
stock_predictions_actual = close_scaler.inverse_transform(stock_predictions)
y_test_stock_actual = close_scaler.inverse_transform(y_test_stock.reshape(-1, 1))

# Метрики
stock_mae = mean_absolute_error(y_test_stock_actual, stock_predictions_actual)
stock_rmse = np.sqrt(mean_squared_error(y_test_stock_actual, stock_predictions_actual))

print('\n=== LSTM Stock Predictions ===')
print(f'MAE: ${stock_mae:.2f}')
print(f'RMSE: ${stock_rmse:.2f}')

# Визуализация
plt.figure(figsize=(15, 6))
plt.plot(range(len(y_test_stock_actual)), y_test_stock_actual, label='Actual Price', linewidth=2)
plt.plot(range(len(stock_predictions_actual)), stock_predictions_actual, label='LSTM Predictions', linewidth=2, linestyle='--', alpha=0.8)
plt.title('Stock Price Prediction with LSTM', fontsize=14, fontweight='bold')
plt.xlabel('Days')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
# Выводы

## 🎯 Главные инсайты:

### 1. Airline Passengers (Univariate):
- **SARIMA** отлично работает для данных с явной сезонностью
- **Prophet** автоматически находит тренды и сезонность
- **LSTM/GRU** требуют больше данных, но могут выучить сложные паттерны

### 2. Stock Prices (Multivariate):
- Финансовые данные очень волатильны
- Технические индикаторы важны для предсказания
- LSTM может использовать множество признаков одновременно

### 3. Сравнение методов:

**Классические (ARIMA, SARIMA, Prophet)**:
- ✅ Быстрые
- ✅ Интерпретируемые
- ✅ Работают на малых данных
- ✅ Хороши для краткосрочных прогнозов
- ❌ Ограничены линейными зависимостями
- ❌ Плохо работают с multivariate данными

**Deep Learning (LSTM, GRU)**:
- ✅ Выучивают сложные нелинейные паттерны
- ✅ Работают с multivariate данными
- ✅ Могут выучить долгосрочные зависимости
- ❌ Требуют много данных
- ❌ Долго обучаются
- ❌ Сложнее интерпретировать
- ❌ Склонны к переобучению

### 4. Рекомендации:

**Используйте ARIMA/SARIMA/Prophet если:**
- Univariate временной ряд
- Малое количество данных (<1000 точек)
- Нужна интерпретируемость
- Явная сезонность

**Используйте LSTM/GRU если:**
- Multivariate временные ряды
- Много данных (>10,000 точек)
- Сложные нелинейные зависимости
- Долгосрочные зависимости

### 5. Следующие шаги:
- Attention механизмы для LSTM
- Transformer для временных рядов
- Ансамбль классических и DL методов
- Аномалия detection во временных рядах

In [ ]:
# Сохранение моделей
import os
os.makedirs('../models', exist_ok=True)

lstm_model.save('../models/lstm_airline.h5')
gru_model.save('../models/gru_airline.h5')
stock_lstm_model.save('../models/lstm_stock.h5')
joblib.dump(scaler, '../models/scaler_airline.pkl')
joblib.dump(scaler_stock, '../models/scaler_stock.pkl')

print('✅ Все модели сохранены в папку models/')